# LangSmith 평가: 에이전트 및 RAG 시스템 평가

이 노트북에서는 **LangSmith를 사용한 체계적인 평가 방법**을 개념적으로 배웁니다.

## 왜 체계적인 평가가 필요할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                    LLM 애플리케이션 평가의 어려움                    │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   전통적인 소프트웨어:                                              │
│   입력 → 함수 → 출력                                               │
│   결과가 결정적 (항상 같은 출력)                                    │
│   테스트: assert output == expected                                │
│                                                                    │
│   LLM 애플리케이션:                                                 │
│   입력 → LLM → 출력                                                │
│   결과가 비결정적 (매번 다른 출력)                                  │
│   "정답"이 명확하지 않음                                           │
│                                                                    │
│   해결: LLM을 평가자로 사용! (LLM-as-Judge)                        │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## LangSmith 평가 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    LangSmith 평가 흐름                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ 데이터셋 생성                                                 │
│   ┌─────────────────────────────────────────┐                      │
│   │ Input (질문)        │ Output (정답)     │                      │
│   ├─────────────────────┼───────────────────┤                      │
│   │ "가장 많이 판매된.." │ "Hot Girl"        │                      │
│   │ "Led Zeppelin..."   │ "14개의 앨범"     │                      │
│   └─────────────────────┴───────────────────┘                      │
│                                                                    │
│   2️⃣ 예측 함수 정의                                                │
│   def predict(example):                                            │
│       return agent.invoke(example["input"])                        │
│                                                                    │
│   3️⃣ 평가자 정의                                                   │
│   def evaluator(run, example):                                     │
│       return {"score": compare(run.output, example.output)}        │
│                                                                    │
│   4️⃣ 평가 실행                                                     │
│   evaluate(predict, data=dataset, evaluators=[evaluator])          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 평가 유형

## 1.1 답변 정확도 평가

In [ ]:
# 의사 코드 - LangSmith 연동 필요

# 답변 정확도 평가자
def answer_evaluator(run, example):
    """
    RAG 답변 정확도 평가
    
    LLM을 사용하여 정답과 예측 답변 비교
    """
    # 입력, 정답, 예측값
    input_question = example.inputs["input"]
    reference = example.outputs["output"]  # 정답
    prediction = run.outputs["response"]   # 모델 응답
    
    # LLM 평가자
    llm = ChatOllama(model="llama3.2", temperature=0)
    
    # 평가 프롬프트
    eval_prompt = f"""
    질문: {input_question}
    정답: {reference}
    학생 답변: {prediction}
    
    학생의 답변이 정답과 의미적으로 일치하면 1, 아니면 0을 반환하세요.
    """
    
    score = llm.invoke(eval_prompt)
    
    return {"key": "answer_accuracy", "score": int(score)}

print("✅ 답변 정확도 평가자 개념 설명 완료")

## 1.2 도구 호출 평가

In [ ]:
# 의사 코드 - 도구 호출 평가

def check_specific_tool_call(run, example):
    """
    첫 번째 도구 호출이 예상대로인지 확인
    """
    expected_tool = 'sql_db_list_tables'
    
    response = run.outputs["response"]
    
    # 도구 호출 가져오기
    try:
        tool_call = response.tool_calls[0]['name']
    except:
        tool_call = None
    
    score = 1 if tool_call == expected_tool else 0
    
    return {"key": "first_tool_call", "score": score}

print("✅ 도구 호출 평가자 개념 설명 완료")

## 1.3 에이전트 경로(Trajectory) 평가

In [ ]:
# 의사 코드 - 에이전트 경로 평가

def contains_all_tool_calls_in_order(run, example):
    """
    에이전트가 예상된 도구들을 올바른 순서로 호출했는지 확인
    """
    expected = [
        'sql_db_list_tables',  # 1. 테이블 목록 조회
        'sql_db_schema',       # 2. 스키마 조회
        'sql_db_query_checker', # 3. 쿼리 검증
        'sql_db_query',        # 4. 쿼리 실행
        'check_result'         # 5. 결과 확인
    ]
    
    messages = run.outputs["response"]
    
    # 실제 도구 호출 추출
    tool_calls = [
        tc['name'] for m in messages 
        for tc in getattr(m, 'tool_calls', [])
    ]
    
    # 순서대로 포함되어 있는지 확인
    it = iter(tool_calls)
    score = 1 if all(elem in it for elem in expected) else 0
    
    return {"key": "trajectory_in_order", "score": score}

print("✅ 에이전트 경로 평가자 개념 설명 완료")

# 2. 평가 데이터셋 예시

In [ ]:
# SQL 에이전트 평가용 데이터셋 예시

sql_evaluation_examples = [
    {
        "input": "어느 나라의 고객이 가장 많이 지출했나요? 그리고 얼마를 지출했나요?",
        "output": "가장 많이 지출한 나라는 미국으로, 총 지출액은 $523.06입니다"
    },
    {
        "input": "2013년에 가장 많이 판매된 트랙은 무엇인가요?",
        "output": "2013년에 가장 많이 판매된 트랙은 Hot Girl입니다."
    },
    {
        "input": "Led Zeppelin 아티스트는 몇 개의 앨범을 발매했나요?",
        "output": "Led Zeppelin은 14개의 앨범을 발매했습니다"
    },
    {
        "input": "'Big Ones' 앨범의 총 가격은 얼마인가요?",
        "output": "'Big Ones' 앨범의 총 가격은 14.85입니다"
    },
    {
        "input": "2009년에 어떤 영업 담당자가 가장 많은 매출을 올렸나요?",
        "output": "Steve Johnson이 2009년에 가장 많은 매출을 올렸습니다"
    },
]

print("=== SQL 에이전트 평가 데이터셋 ===")
for i, ex in enumerate(sql_evaluation_examples):
    print(f"\n[예제 {i+1}]")
    print(f"  질문: {ex['input']}")
    print(f"  정답: {ex['output']}")

# 3. 평가 실행 (개념)

```python
from langsmith import Client
from langsmith.evaluation import evaluate

# 1. LangSmith 클라이언트 생성
client = Client()

# 2. 데이터셋 생성
dataset = client.create_dataset("sql-agent-eval")
client.create_examples(
    inputs=[{"input": ex["input"]} for ex in examples],
    outputs=[{"output": ex["output"]} for ex in examples],
    dataset_id=dataset.id
)

# 3. 예측 함수
def predict(example):
    result = agent.invoke({"messages": [("user", example["input"])]})
    return {"response": result["messages"][-1].content}

# 4. 평가 실행
results = evaluate(
    predict,
    data="sql-agent-eval",
    evaluators=[answer_evaluator, trajectory_evaluator],
    num_repetitions=3,  # 3번 반복 실행
    experiment_prefix="sql-agent-v1"
)
```

---

## 정리: LangSmith 평가

### 평가 유형

| 평가 유형 | 측정 대상 | 평가 방법 |
|----------|----------|----------|
| **답변 정확도** | 최종 응답 | LLM-as-Judge |
| **도구 호출** | 올바른 도구 선택 | 직접 비교 |
| **에이전트 경로** | 도구 호출 순서 | 순서 비교 |

### 핵심 API

```python
# LangSmith 평가
from langsmith.evaluation import evaluate

results = evaluate(
    predict_fn,           # 예측 함수
    data=dataset_name,    # 데이터셋
    evaluators=[...],     # 평가자 목록
    num_repetitions=3,    # 반복 횟수
)
```

### 평가자 구조

```python
def evaluator(run, example) -> dict:
    # run.outputs: 모델 출력
    # example.inputs: 입력
    # example.outputs: 정답
    
    score = compute_score(...)
    
    return {
        "key": "metric_name",
        "score": score  # 0 또는 1
    }
```

## ch10 요약: 평가

| 기능 | 용도 |
|------|------|
| **검색 관련성 평가** | RAG 문서 필터링 |
| **답변 정확도** | 최종 응답 품질 |
| **도구 호출 평가** | 에이전트 동작 검증 |
| **경로 평가** | 에이전트 추론 과정 검증 |

## 전체 커리큘럼 요약

| 챕터 | 주제 |
|------|------|
| ch01 | LangChain 기초, 프롬프트, 체인 |
| ch02 | RAG 기초, 벡터 DB |
| ch03 | 고급 RAG, Agentic RAG |
| ch04 | 메모리 관리 |
| ch05 | LangGraph 기본 챗봇 |
| ch06 | 에이전트와 도구 |
| ch07 | 고급 패턴 (Reflection, Subgraph, Supervisor) |
| ch08 | 고급 기능 (Streaming, Interrupt, State) |
| ch09 | 배포 |
| ch10 | 평가 |